#### BEFORE YOU BEGIN: Please note that this is an example notebook that will run against your Fiddler Trial environment as-is.  However, if you want this notebook to register details about your existing SageMaker model, you will need to update the sections below as called out by the comments below.

## Install the Fiddler Client and Import Libraries

In [ ]:
# Install the fiddler client
!pip install fiddler-client

In [ ]:
# Import the needed libraries
import fiddler as fdl
import pandas as pd

In [ ]:
# Output version of the Fiddler client
fdl.__version__

## Connect the Fiddler Client to Your Trial Environment

In [ ]:
# Please update these variables with those specific to your Fiddler Trial Environment

URL = 'https://yourcompany.try.fiddler.ai'
ORG_ID = 'yourcompany'
AUTH_TOKEN = 'UUID-Token-Here-Found-In-Your-Fiddler-Env-Settings-Page'

client = fdl.FiddlerApi(URL, ORG_ID, AUTH_TOKEN)

## Set Variables and Create a Fiddler Project

In [ ]:
# Create Project.  You can keep these variables as is or name them specific to the SageMaker model you're registering.
PROJECT_ID = 'credit_default'
DATASET_ID = 'credit_default_dataset'
MODEL_ID = 'credit_default_model'

client.create_project(PROJECT_ID)

## Read in the Model's Training Data and Upload it to Fiddler

In [28]:
# This is an example baseline dataset.  Feel free to update this with your own training dataset for your SageMaker model.  
# If you update this with your own training dataset, you will need to update the model info step below to specify the features, output, target, etc. for your model too.

df_baseline = pd.read_csv('https://raw.githubusercontent.com/fiddler-labs/fiddler-samples/master/sagemaker/train.csv')

In [ ]:
dataset_info = fdl.DatasetInfo.from_dataframe(df_baseline, max_inferred_cardinality=1000)
dataset_info

In [ ]:
upload_result = client.upload_dataset(PROJECT_ID,
                                      dataset={'baseline': df_baseline},
                                      dataset_id=DATASET_ID,
                                      info=dataset_info)

## Create a Model Info Object and Register Your Model's Metadata with Fiddler

In [ ]:
# Note: If you didn't update the training dataset read in above, you can leave this cell alone.  However, if you
# updated the training dataset above to be your own, you will need to update this cell to specify your model's own features, output, target, etc.

# Specify task
model_task = 'binary'

if model_task == 'regression':
    model_task_fdl = fdl.ModelTask.REGRESSION
    
elif model_task == 'binary':
    model_task_fdl = fdl.ModelTask.BINARY_CLASSIFICATION

elif model_task == 'multiclass':
    model_task_fdl = fdl.ModelTask.MULTICLASS_CLASSIFICATION

elif model_task == 'ranking':
    model_task_fdl = fdl.ModelTask.RANKING

    
# Specify column types
target = 'TARGET'
outputs = ['prediction']
features = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 
                  'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
                  'DAYS_REGISTRATION', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 
                  'REGION_RATING_CLIENT_W_CITY', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY',
                  'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
                  'AMT_REQ_CREDIT_BUREAU_YEAR', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
                  'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG']
     
# Generate a Fiddler ModelInfo object
model_info = fdl.ModelInfo.from_dataset_info(
    dataset_info=dataset_info,
    dataset_id=DATASET_ID,
    model_task=model_task_fdl,
    target=target,
    outputs=outputs,
    features=features,
    binary_classification_threshold=.125,
    description='This model predicts the probability of a customer defaulting on their home loan',
    display_name='XGBoost Credit Default Detection Model'
)
model_info

In [ ]:
# Register the model with Fiddler
client.register_model(
    project_id=PROJECT_ID,
    dataset_id=DATASET_ID,
    model_id=MODEL_ID,
    model_info=model_info
)